In [ ]:
!pip install catboost

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [18]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [19]:
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']

  submit1 = pd.read_csv('0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv('1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv('2_fold' + str(r) + '.csv')

  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')

  test.columns = ['target', 'ID X PCODE', 'Label1', 'Label2', 'Label3']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

In [20]:
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

submit1 = pd.read_csv('0_main.csv')
submit2 = pd.read_csv('1_main.csv')
submit3 = pd.read_csv('2_main.csv')

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1', 'Label2', 'Label3']

In [21]:
from catboost import CatBoostClassifier

models = []

models.append(CatBoostClassifier(random_state=21, max_depth=3))
models.append(CatBoostClassifier(random_state=21, max_depth=4))
models.append(CatBoostClassifier(random_state=21, max_depth=5))

In [22]:
for i in range(len(models)):
  models[i].fit(fold_train.drop(columns=['target', 'ID X PCODE']), fold_train['target'])

Learning rate set to 0.219943
0:	learn: 0.2590624	total: 399ms	remaining: 6m 38s
1:	learn: 0.1232427	total: 756ms	remaining: 6m 17s
2:	learn: 0.0760826	total: 1.12s	remaining: 6m 13s
3:	learn: 0.0508175	total: 1.46s	remaining: 6m 3s
4:	learn: 0.0438284	total: 1.78s	remaining: 5m 53s
5:	learn: 0.0388291	total: 2.1s	remaining: 5m 47s
6:	learn: 0.0366275	total: 2.41s	remaining: 5m 42s
7:	learn: 0.0354392	total: 2.73s	remaining: 5m 37s
8:	learn: 0.0341498	total: 3.04s	remaining: 5m 34s
9:	learn: 0.0334244	total: 3.37s	remaining: 5m 34s
10:	learn: 0.0331505	total: 3.73s	remaining: 5m 35s
11:	learn: 0.0331003	total: 4.02s	remaining: 5m 30s
12:	learn: 0.0328498	total: 4.33s	remaining: 5m 29s
13:	learn: 0.0326522	total: 4.65s	remaining: 5m 27s
14:	learn: 0.0326423	total: 4.96s	remaining: 5m 26s
15:	learn: 0.0326142	total: 5.27s	remaining: 5m 24s
16:	learn: 0.0326091	total: 5.56s	remaining: 5m 21s
17:	learn: 0.0325866	total: 5.88s	remaining: 5m 21s
18:	learn: 0.0325312	total: 6.21s	remaining: 5

In [23]:
predicts = []
for i in range(len(models)):
  predicts.append(models[i].predict_proba(test.drop(columns=['ID X PCODE']))[:, 1])

In [24]:
test['target'] = np.mean(predicts, axis=0)

In [25]:
true_values = []
for v in test_.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test_.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [26]:
test = test[['ID X PCODE', 'target']]
test.columns = ['ID X PCODE', 'Label']

In [27]:
for v in true_values:
  test = test.append({'ID X PCODE':v,'Label':1.0}, ignore_index=True)

In [29]:
test.reset_index(drop=True, inplace=True)
test.to_csv('submission_stack2.csv', index=False)

OK